In [1]:
import pandas as pd
import numpy as np
import math 
import os 
from datetime import datetime, timedelta
import pymap3d # python3 -m pip install pymap3d
import random
import pickle
from scipy.special import lambertw
from scipy import stats
from scipy.interpolate import interp1d as inte1
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output, save
from bokeh.layouts import gridplot, row, column
from bokeh.models import Range1d
output_notebook() 
# output_notebook() 
import matplotlib as mpl
import tikzplotlib 
save_data_bool = False # Save data or not

Loading BokehJS ...

## Load data

In [2]:
#sol_file='oilrag/databuild_Hp4'
Hp=4
user='oilrag'
sol_file=user+'/databuild_Hp'+str(Hp)

save_data = pickle.load( open( sol_file+"/save_data_online.pkl", "rb" ) )
Time=save_data['Time']
xs=save_data['xs']
ys=save_data['ys']
bs=save_data['bs']

# Vector for plotting, removed the non-transmitted instants
Timep = Time[bs != 0]
xsp = xs[bs != 0] 
ysp = ys[bs != 0] 

fig = figure(plot_width=900, plot_height=450)
#fig.line(Timer, xr, line_color='navy', legend_label="x", alpha=0.2)
#fig.circle(Timer, xr, line_color='navy', legend_label="x", fill_alpha=0.2, alpha=0.2)
fig.line(Timep, xsp, line_color='navy', legend_label="x")
fig.circle(Timep, xsp, line_color='navy', color = 'cyan', legend_label="x")
#fig.line(Timer, yr, line_color='red', legend_label="y", alpha=0.2)
#fig.circle(Timer, yr, line_color='red', legend_label="y", fill_alpha=0.2, alpha=0.2)
fig.line(Timep, ysp, line_color='red', legend_label="y")
fig.circle(Timep, ysp, line_color='red', color = 'pink', legend_label="y")
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "y"
show(fig)

fig = figure(plot_width=900, plot_height=450)
fig.line(xsp, ysp, line_color='navy', legend_label="p")
fig.circle(xsp, ysp, color='navy', legend_label="p")
#fig.line(xr, yr, line_color='navy', legend_label="p raw", alpha=0.2)
#fig.circle(xr, yr, color='navy', legend_label="p raw", fill_alpha=0.2, alpha=0.2)
fig.xaxis.axis_label = "x"
fig.yaxis.axis_label = "y"
show(fig)

Data_real=pickle.load( open( sol_file+"/Data_real_online.pkl", "rb" ) )

In [3]:
sol_newm=sol_file+'/sol_online'

Data_sol_p = {} # wc method
Data_sol_mpc = {} # mpc

lower_bounds=np.loadtxt(sol_newm+'/lb.txt')

for p_lb in lower_bounds:
    Data_sol_p[p_lb]      =pd.read_csv(sol_newm+'/sol_h_'+str(Hp)+'_lb_'+str(int(p_lb))+'_mode2.csv')
    Data_sol_mpc[p_lb]    =pd.read_csv(sol_newm+'/sol_h_'+str(Hp)+'_lb_'+str(int(p_lb))+'_mpc.csv')
    
Data_GeoI =pd.read_csv(sol_file+'GeoI.csv')  

In [4]:
p_real=Data_real.loc[Data_real.b>0].priv.mean()
p_GeoI=Data_GeoI.loc[Data_real.b>0].priv.mean()
print("Privacy real : ",p_real )
print("Privacy Geo-I: ",p_GeoI )
plot_pr_b=[]
plot_pr_b_perc=[]
plot_pr_mpc=[]
plot_pr_mpc_perc=[]


for p_lb in lower_bounds:
    #load privacy
    p_b      =Data_sol_p[p_lb].loc[Data_sol_p[p_lb].tra>0].priv.mean() #comparing just transmitted points
    p_mpc    =Data_sol_mpc[p_lb].loc[Data_sol_mpc[p_lb].util>0].priv.mean()#comparing just transmitted points 
    plot_pr_b+=[p_b]
    plot_pr_b_perc+=[(p_b-p_real)/p_real]
    plot_pr_mpc+=[p_mpc]
    plot_pr_mpc_perc+=[(p_mpc-p_real)/p_real]
    
    #load utility
    util_sol=Data_sol_p[p_lb].loc[Data_sol_p[p_lb].tra>0].util.mean()

    
    
    print("Privacy bound "+str(p_lb)+"  : ",p_b," ",(p_b-p_real)/p_real,' utility mean ', util_sol)
    print("Privacy mpc             : ",p_mpc," ",(p_mpc-p_real)/p_real,'\n')
    
    fail1=len(Data_sol_p[p_lb].loc[(Data_sol_p[p_lb].tra>0) & (Data_sol_p[p_lb].priv<p_lb-10**-1)])
    total=len(Data_sol_p[p_lb].loc[(Data_sol_p[p_lb].tra>0)])
    print('times restriction unfulfilled   :',fail1,'fraction ',fail1/total,'\n')
    print("resolution average time    :",Data_sol_p[p_lb].loc[Data_sol_p[p_lb].tra>0].time.mean(),'s')
    print("resolution average time mpc:",Data_sol_mpc[p_lb].loc[Data_sol_mpc[p_lb].util>0].time.mean(),'s\n\n')
    
    

Privacy real :  5815745.097692999
Privacy Geo-I:  6303712.008531598
Privacy bound 500000.0  :  6137799.810859347   0.05537634606683518  utility mean  253.6153088475223
Privacy mpc             :  6120278.94315365   0.05236368519339914 

times restriction unfulfilled   : 1 fraction  0.006944444444444444 

resolution average time    : 0.08349176413483089 s
resolution average time mpc: 0.4758819225761626 s


Privacy bound 750000.0  :  6271850.631441247   0.07842598430408797  utility mean  368.6182859085793
Privacy mpc             :  6257110.74980692   0.0758915056798144 

times restriction unfulfilled   : 3 fraction  0.020833333333333332 

resolution average time    : 0.0934309677945243 s
resolution average time mpc: 0.44301528731981915 s


Privacy bound 1000000.0  :  6465300.529944186   0.11168911658608524  utility mean  478.7469758119221
Privacy mpc             :  6440133.484630527   0.10736171830935498 

times restriction unfulfilled   : 2 fraction  0.013888888888888888 

resolution ave

## Privacy mean vs privacy the lower bound

In [5]:
fig = figure(plot_width=900, plot_height=300)
fig.line(lower_bounds, plot_pr_b, line_color='red', legend_label="priv new method")
fig.circle(lower_bounds, plot_pr_b, color='red', legend_label="priv new method")

fig.line(lower_bounds, plot_pr_mpc, line_color='blue', legend_label="priv mpc-1")
fig.circle(lower_bounds, plot_pr_mpc, color='blue', legend_label="priv mpc-1")
fig.line(lower_bounds, p_real*np.ones(len(lower_bounds)), line_color='black', legend_label="priv real")
fig.line(lower_bounds, p_GeoI*np.ones(len(lower_bounds)), line_dash='dashed',line_color='grey', legend_label="priv GeoI")
fig.xaxis.axis_label = "bounds"
fig.yaxis.axis_label = "privacy"
#fig.y_range = Range1d(500, 1200)
fig.legend.location = "bottom_right"
show(fig)

fig = figure(plot_width=900, plot_height=300)
fig.line(lower_bounds, plot_pr_b_perc, line_color='red', legend_label="priv new method")
fig.circle(lower_bounds, plot_pr_b_perc, color='red', legend_label="priv new method")


fig.line(lower_bounds, plot_pr_mpc_perc, line_color='blue', legend_label="priv mpc-1")
fig.circle(lower_bounds, plot_pr_mpc_perc, color='blue', legend_label="priv mpc-1")

fig.line(lower_bounds, (p_GeoI-p_real)/p_real*np.ones(len(lower_bounds)), line_dash='dashed',line_color='grey', legend_label="priv GeoI")
fig.xaxis.axis_label = "bounds"
fig.yaxis.axis_label = "percentage variation"
#fig.y_range = Range1d(0, 1)
fig.legend.location = "bottom_right"
show(fig)

In [6]:
# Ploting privacy curve 
inicio=0
Timep2=save_data['Time'][inicio:len(xs)-Hp].loc[save_data['bs']>0]
fig = figure(plot_width=900, plot_height=300)
fig.line(Timep, Data_real.loc[Data_real.b>0].priv, line_color='black', legend_label="priv real")
fig.circle(Timep, Data_real.loc[Data_real.b>0].priv, color='black', legend_label="priv real")
fig.line(Timep, Data_GeoI.loc[Data_real.b>0].priv,line_dash='dashed', line_color='grey', legend_label="priv Geo-I")

#Choose lower bound to plot
forlb=1000000

for lb in [forlb]:
    fig.line(Timep2, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].priv, line_color='red', legend_label="priv wc method "+str(lb))
    fig.circle(Timep2, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].priv, color='red', legend_label="priv wc method "+str(lb))
    
    fig.line(Timep[:-Hp+1], Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].priv, line_color='blue', legend_label="priv mpc",alpha=0.5)
    fig.circle(Timep[:-Hp+1], Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].priv, color='blue', legend_label="priv mpc",alpha=0.25)
    
    fig.line(Timep2, lb*np.ones(len(Timep2)), line_color='red', legend_label="limit "+str(lb),line_dash=[6,3])
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "privacy"
fig.y_range = Range1d(0, forlb*2)
show(fig)


#Utility
fig = figure(plot_width=900, plot_height=250)
for lb in [forlb]:
    fig.line(Timep2, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].util, line_color='red', legend_label="util wc method "+str(lb))
    fig.circle(Timep2, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].util, color='red', legend_label="util wc method "+str(lb))
fig.circle(Timep, Data_GeoI.loc[Data_real.b>0].util, line_dash='dashed', line_color='grey', legend_label="util Geo-I")  
show(fig)
    
#Position
fig = figure(plot_width=900, plot_height=500)
fig.line(Data_real.loc[Data_real.b>0].x, Data_real.loc[Data_real.b>0].y, line_color='black', legend_label="real")
fig.circle(Data_real.loc[Data_real.b>0].x, Data_real.loc[Data_real.b>0].y, color='black', legend_label="real")
fig.line(Data_GeoI.loc[Data_real.b>0].x, Data_GeoI.loc[Data_real.b>0].y,line_dash='dashed', line_color='grey', legend_label="Geo-I")
fig.circle(Data_GeoI.loc[Data_real.b>0].x, Data_GeoI.loc[Data_real.b>0].y,line_dash='dashed', color='grey', legend_label="Geo-I")
for lb in [forlb]:
    fig.line(Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].x, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].y, line_color='red', legend_label="wc method ")
    fig.circle(Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].x, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].y, color='red', legend_label="wc method ")
    
    fig.line(Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].x, Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].y, line_color='blue', legend_label="mpc",alpha=0.5)
    fig.circle(Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].x, Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].y, color='blue', legend_label="mpc",alpha=0.25)
fig.xaxis.axis_label = "x"
fig.yaxis.axis_label = "y"
show(fig)
